<a href="https://colab.research.google.com/github/felipeaguirre66/RealStateScrapper/blob/main/query_real_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [139]:
# Installs for Google Sheet connection
import gspread
import google.auth
from google.colab import auth
from google.colab import files
from google.auth import default
!pip install --upgrade -q pygsheets
import pygsheets

# Authenticate in drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Other imports
!pip install unidecode
import unidecode
import pandas as pd

# Authenticate in drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Helpers
def open_sheet(link_db):
  gc = pygsheets.client.Client(creds)
  sh = gc.open_by_url(link_db)
  worksheet = sh.worksheets()[-1]

  # Read all the data from the worksheet
  data = worksheet.get_all_values(returnas='matrix')

  # Load the data into a pandas DataFrame
  df = pd.DataFrame(data[1:], columns=data[0])

  return df

def preprocess_df(df):
  df_raw = df.copy()
  df_raw['city'] = df_raw['city'].apply(lambda x: unidecode.unidecode(x.lower().strip()))
  df_raw['adress'] = df_raw['adress'].apply(lambda x: unidecode.unidecode(x.lower().strip()))

  df_raw['rooms'] = df_raw['rooms'].astype(float)
  df_raw['squared_meters'] = df_raw['squared_meters'].astype(float)
  df_raw['buy_rent_price'] = df_raw['buy_rent_price'].astype(float)
  df_raw['expensas_price'] = df_raw['expensas_price'].astype(float)
  df_raw['total_price'] = df_raw['total_price'].astype(float)

  return df_raw

# Query Database

In [3]:
link_db = 'https://docs.google.com/spreadsheets/d/1H3D2xBKoXIxh3k-gJqjXmWC-50fNUT58lXutkZhwo20/edit#gid=0'

In [140]:
df = open_sheet(link_db) # Read df

In [141]:
pp_df = preprocess_df(df) # Preprocess df

In [196]:
query_dict = {
              'service':[''],
              'city':[''],
              'adress':[''],
              'property_type':['casa'],
              'contract_type':['venta'],
              'rooms':[[3,6]],
              'squared_meters':[[200,1000]],
              'buy_rent_currency':['USD'],
              'expensas_currency':[],
              'buy_rent_price':[],
              'expensas_price':[],
              'total_price':[[0,50000]]
            }

In [197]:
df_query = pp_df.copy()
for key, value in query_dict.items():
  indices_coincidencia = []
  if len(value)>0:

    if type(value[0])==str: # String columns
      for query_val in value:
        indices_coincidencia.extend([i for i,x in enumerate(df_query[key].values) if query_val in x])
      df_query = df_query.iloc[indices_coincidencia]
    
    else:
      for query_val in value:
        indices_coincidencia.extend([i for i,x in enumerate(df_query[key].values) if x >= query_val[0] and x <= query_val[1]])
      df_query = df_query.iloc[indices_coincidencia]

In [204]:
df_query[['link','city','service','adress','property_type','rooms','squared_meters','buy_rent_currency','total_price']]

,link,city,service,adress,property_type,rooms,squared_meters,buy_rent_currency,total_price
233,https://casa.mercadolibre.com.ar/MLA-134768016...,"calle 58 1405, mar del tuyu, provincia de buen...",mercado_libre,calle 58 1405,casa,2.0,40.0,USD,25000.0
240,https://casa.mercadolibre.com.ar/MLA-134626613...,"presidente nestor kirchner 11300, carlos spega...",mercado_libre,presidente nestor kirchner 11300,casa,3.0,80.0,USD,46000.0


In [167]:
df.columns

Index(['link', 'service', 'city', 'adress', 'property_type', 'contract_type',
       'rooms', 'squared_meters', 'buy_rent_currency', 'expensas_currency',
       'buy_rent_price', 'expensas_price', 'total_price', '', '', '', '', '',
       '', '', '', '', '', '', '', ''],
      dtype='object')

In [175]:
df.query('')

'NaN'